In [1]:
import pandas as pd
import matplotlib.pyplot as plt

#makes graph interactive
%matplotlib qt 

In [2]:
#used when generating the data.
num_nodes = [[30, 30,30], [35, 35,35], [40, 40,40], [45, 45,45], [50, 50,50], [55, 55,55], [60, 60,60], [65, 65,65], [70, 70,70]]
lr = [0.0001, 0.001, 0.005, 0.01, 0.025, 0.05, 0.1]
fractions = [10, 20, 30, 40, 50, 60, 70, 80, 90]
activations = ["elu", "relu", "go", "sigmoid", "tanh"]
datasets = [ ("CSR3", "CSR1"), ("CSR3", "CSR2"), ("CSR3", "S2"), ("S2", "CSR1"), ("S2", "CSR3"), ("SYS2", "CSR1"), ("SYS2", "CSR3")  ]
datasets_single =["SYS1", "SYS2", "CSR3", "CSR2", "S2"]



In [3]:
#reads in all data sets. 
df_lr_nodes =  pd.read_excel("./results/lr_node_csr3.xlsx", engine = "openpyxl")
df_dlb =  pd.read_csv("./results/dlb_results.csv")


df02 = pd.read_csv("./results/elu_dense_lr2.csv")

# df_small_sys1 = pd.read_csv("./results/sys1_small_step_results.csv")
# df_small_sys2 = pd.read_csv("./results/sys2_small_step_results.csv")
# go_small_sys1 = pd.read_excel("./results/GO_PSSE_SYS1.xlsx", engine = "openpyxl", sheet_name="SYS1")
# go_small_sys2 = pd.read_excel("./results/GO_PSSE_SYS1.xlsx", engine = "openpyxl", sheet_name="SYS2")

ex = pd.ExcelFile("./results/small_steps.xlsx", engine = "openpyxl")
df_small = {}
for d in datasets_single:
    df_small[d] = pd.read_excel(ex, sheet_name= "{}".format(d))



ex = pd.ExcelFile("./results/transfer_results.xlsx", engine = "openpyxl")
df_tran = {}
for d in datasets:
    df_tran[d] = pd.read_excel(ex, sheet_name= "{}, {}".format(d[0], d[1]))

# ex = pd.ExcelFile("./results/incremental_results_2_scratch.xlsx", engine = "openpyxl")
# df_scra = {}
# for a in activations:
#     df_scra[a] = pd.read_excel(ex, sheet_name=a)
    
    

#scratch
ex = pd.ExcelFile("./results/transfer_results_scratch.xlsx", engine = "openpyxl")
df_scra = {}
for d in datasets:
    df_scra[d] = pd.read_excel(ex, sheet_name= "{}, {}".format(d[0], d[1])) 
    
    
# ex = pd.ExcelFile("./results/incremental_results_midpoints.xlsx", engine = "openpyxl")
# df_inc = {}
# for d in datasets:
#     df_inc[d] = pd.read_excel(ex, sheet_name= "{}, {}".format(d[0], d[1])) 
 


# #incremental
# ex = pd.ExcelFile("./results/csr3_csr1_incremental_results_backtracking2.xlsx", engine = "openpyxl")
# df_inc2 = {}
# for a in activations:
#     df_inc2[a] = pd.read_excel(ex, sheet_name=a)
#     break


# df1 = pd.read_excel("./results/elu_lr_nodes.xlsx", sheet_name="elu_single_layer", engine="openpyxl")
# df2 = pd.read_excel("./results/elu_lr_nodes.xlsx", sheet_name="elu_double_layer", engine="openpyxl")
# df3 = pd.read_excel("./results/elu_lr_nodes.xlsx", sheet_name="elu_triple_layer", engine="openpyxl")
# df_go = pd.read_excel("./results/GO_results.xlsx", sheet_name="SYS1_80_20", engine="openpyxl")


ex = pd.ExcelFile("./results/80_20_results_all.xlsx", engine = "openpyxl")
df_80_20 = {}
for d in datasets_single:
    df_80_20[d] = pd.read_excel(ex, sheet_name=d)
    
df_go = pd.read_excel("./results/go_csr3_results.xlsx", engine="openpyxl")



df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR3", engine="openpyxl")
df_sys2 = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR2", engine="openpyxl")

In [4]:
#This code is for the stepping calculations

#calculates predictive mean squared error. Pass in all data and train split size
def pmse(y_predicted, y_actual, split = 0.8):
    pmse_list = []
    for i in range(0, len(y_predicted.keys())):  
        pmse =0;
        split_fraction = fractions[i]/100
        for a, b in zip(y_predicted[y_predicted.keys()[i]][int(len(y_actual) * split_fraction):int(len(y_actual) * (split_fraction + 10/100))], list(y_actual[int(len(y_actual) * split_fraction):int(len(y_actual) * (split_fraction + 10/100))])):
#             print(a,b)
            pmse += (a-b)**2
        print(len(y_predicted[y_predicted.keys()[i]][int(len(y_predicted.values) * split_fraction):int(len(y_predicted.values) * (split_fraction + 10/100))]))
        pmse_list.append(pmse/len(y_predicted[y_predicted.keys()[i]][int(len(y_predicted.values) * split_fraction):int(len(y_predicted.values) * (split_fraction + 10/100))]))
    return pmse_list
        
#calculates mean squared error
def mse(y_predicted, y_actual):
    mse_list = []
    for i in range(0, len(y_predicted.keys())):    
        mse =0;
        split_fraction = fractions[i]/100
        for a, b in zip(y_predicted[y_predicted.keys()[i]][:int(len(y_actual) * (split_fraction + 10/100))], list(y_actual[:int(len(y_actual) * (split_fraction + 10/100))])):
#             print(a, b)
            mse += (a-b)**2
        mse_list.append(mse/int(len(y_actual) * (split_fraction + 10/100)))
    return mse_list

#calculates Mean Absolute Percentage Error
def mape(y_predicted, y_actual):
    mape_list = []
    for i in range(0, len(y_predicted.keys())):    
        mape =0;
        split_fraction = fractions[i]/100
        for a, b in zip(y_predicted[y_predicted.keys()[i][:int(len(y_actual) * (split_fraction + 10/100))]], list(y_actual[:int(len(y_actual) * (split_fraction + 10/100))])):
            mape += abs((a-b)/b)
        mape_list.append(100 * mape/int(len(df2) * (split_fraction + 10/100)))
    return mape_list

#calculates predictive power. Pass in all data and train split size
def pp(y_predicted, y_actual, split = 0.8):
    pmse_list = []
    for i in range(0, len(y_predicted.keys())):  
        pmse =0;
        split_fraction = fractions[i]/100
        for a, b in zip(y_predicted[y_predicted.keys()[i]][int(len(y_actual) * split_fraction):int(len(y_actual) * (split_fraction + 10/100))], list(y_actual[int(len(y_actual) * split_fraction):int(len(y_actual) * (split_fraction + 10/100))])):
    #         print(a,b)
            pmse += ((a-b)/b)**2
        pmse_list.append(pmse/len(y_predicted[y_predicted.keys()[i]][int(len(y_actual) * split_fraction):int(len(y_actual) * (split_fraction + 10/100))]))

    return pmse_list

#calculates Predictive Ratio Risk. Pass in all data and train split size
def prr(y_predicted, y_actual, split = 0.8):
    pmse_list = []
    for i in range(0, len(y_predicted.keys())):  
        pmse =0
        split_fraction = fractions[i]/100
        for a, b in zip(y_predicted[y_predicted.keys()[i]][int(len(y_actual) * split_fraction):int(len(y_actual) * (split_fraction + 10/100))], list(y_actual[int(len(y_actual) * split_fraction):int(len(y_actual) * (split_fraction + 10/100))])):
    #         print(a,b)
            pmse += ((a-b)/a)**2
#             print(a, b)
        pmse_list.append(pmse/len(y_predicted[y_predicted.keys()[i]][int(len(y_actual) * split_fraction):int(len(y_actual) * (split_fraction + 10/100))]))
    return pmse_list


In [9]:
#calculates predictive mean squared error. Pass in all data and train split size
def pmse(y_predicted, y_actual, split = 0.8):
    pmse_list = []
    for i in range(0, len(y_predicted.keys())):  
        pmse =0
#         print(y_predicted[y_predicted.keys()[i]][int(split * len(y_predicted)):])
        for a, b in zip(y_predicted[y_predicted.keys()[i]][int(split * len(y_predicted)):], list(y_actual[int(split* len(y_predicted)):])):
#             print(a,b)
            pmse += (a-b)**2
#         print(len(list(y_actual[int(split* len(y_actual)):])))
        pmse_list.append(pmse/len(list(y_actual[int(split* len(y_actual)):])))
    return pmse_list
        
#calculates mean squared error
def mse(y_predicted, y_actual):
    mse_list = []
    for i in range(0, len(y_predicted.keys())):    
        mse =0;
        for a, b in zip(y_predicted[y_predicted.keys()[i]], list(y_actual)):
#             print(a, b)
            mse += (a-b)**2
        mse_list.append(mse/len(y_actual))
    return mse_list

#calculates Mean Absolute Percentage Error
def mape(y_predicted, y_actual):
    mape_list = []
    for i in range(0, len(y_predicted.keys())):    
        mape =0;
        for a, b in zip(y_predicted[y_predicted.keys()[i]], list(y_actual)):
            mape += abs((a-b)/b)
        mape_list.append(100 * mape/len(y_actual))
    return mape_list

#calculates predictive power. Pass in all data and train split size
def pp(y_predicted, y_actual, split = 0.8):
    pmse_list = []
    for i in range(0, len(y_predicted.keys())):  
        pmse =0;
        for a, b in zip(y_predicted[y_predicted.keys()[i]][int(split* len(y_predicted[y_predicted.keys()[i]])):], list(y_actual[int(split* len(y_actual)):])):
    #         print(a,b)
            pmse += ((a-b)/b)**2
        pmse_list.append(pmse)
    return pmse_list

#calculates Predictive Ratio Risk. Pass in all data and train split size
def prr(y_predicted, y_actual, split = 0.8):
    pmse_list = []
    for i in range(0, len(y_predicted.keys())):  
        pmse =0;
        for a, b in zip(y_predicted[y_predicted.keys()[i]][int(split* len(y_predicted[y_predicted.keys()[i]])):], list(y_actual[int(split* len(y_actual)):])):
            pmse += ((a-b)/a)**2
#             print(a, b)
#             print(pmse)
        pmse_list.append(pmse)

    return pmse_list

In [44]:
#this code for the small step size psme


#calculates predictive mean squared error. Pass in all data and train split size
def pmse(y_predicted, y_actual):
    pmse_list = pd.DataFrame()
    splits = list(range(int(0.8 * len(y_actual)) + 1, len(y_actual)))
    act = ""
    j = 0
    for i in range(0, len(y_predicted.keys())):  
        if (i) % len(splits) == 0:
            act = activations[j]
            pmse_list[act] = [0.0]*len(splits)
            j = j + 1
        pmse = 0
        for a, b in zip(y_predicted[y_predicted.keys()[i]][splits[(i)% len(splits)]:], list(y_actual[splits[(i)% len(splits)]:])):
            pmse += (a-b)**2
#         print(pmse/len(list(y_actual[splits[(i)% len(splits)]:])), len(list(y_actual[splits[(i)% len(splits)]:])) )
        print(pmse/float(len(list(y_actual[splits[(i)% len(splits)]:]))))
        pmse_list[act][i % len(splits)] = pmse/float(len(list(y_actual[splits[(i)% len(splits)]:])))
    return pmse_list
        
#calculates mean squared error
def mse(y_predicted, y_actual):
    mse_list = pd.DataFrame()
    splits = list(range(int(0.8 * len(y_actual)) + 1, len(y_actual)))
    j = 0
    print(y_predicted.keys())
    for i in range(0, len(y_predicted.keys())):   
        if (i) % len(splits) == 0:
            act = activations[j]
            mse_list[act] = [0.0]* len(splits)
            j = j + 1
        mse =0;
        for a, b in zip(y_predicted[y_predicted.keys()[i]], list(y_actual)):
            print(a, b)
            mse += (a-b)**2
        mse_list[act][i%len(splits)] = mse/len(y_actual)
    return mse_list

#calculates Mean Absolute Percentage Error
def mape(y_predicted, y_actual):
    mape_list = []
    for i in range(0, len(y_predicted.keys())):    
        mape =0;
        for a, b in zip(y_predicted[y_predicted.keys()[i]], list(y_actual)):
            mape += abs((a-b)/b)
        mape_list.append(100 * mape/len(y_actual))
    return mape_list

#calculates predictive power. Pass in all data and train split size
def pp(y_predicted, y_actual):
    pmse_list = []
    splits = range(int(len(y_predicted) - len(y_predicted) *0.2), len(y_predicted))

    for i in range(0, len(y_predicted.keys())):  
        pmse =0;
        for a, b in zip(y_predicted[y_predicted.keys()[i]][splits[i]:], list(y_actual[splits[i]:])):

    #         print(a,b)
            pmse += ((a-b)/b)**2
        pmse_list.append(pmse)
    return pmse_list

#calculates Predictive Ratio Risk. Pass in all data and train split size
def prr(y_predicted, y_actual):
    pmse_list = []
    splits = range(int(len(y_predicted) - len(y_predicted) *0.2), len(y_predicted))
    for i in range(0, len(y_predicted.keys())):  
        pmse =0;
        for a, b in zip(y_predicted[y_predicted.keys()[i]][splits[i]:], list(y_actual[splits[i]:])):

            pmse += ((a-b)/a)**2
#             print(a, b)
#             print(pmse)
        pmse_list.append(pmse)

    return pmse_list

In [6]:
p = pmse(df_small_sys1[df_small_sys1.keys()[1:int(len(df_small_sys1)*0.2)+2]], df["FN"])
m = mse(df_small_sys1[df_small_sys1.keys()[1:int(len(df_small_sys1)*0.2)+2]], df["FN"])


In [18]:
i = 0
from matplotlib import lines #marker=markers[4]
from matplotlib import markers
linestyles = list(lines.lineStyles.keys())
markers = list(markers.MarkerStyle.markers.keys())

# csr3_csr1_scratch1, csr_incremental_results_scratch2
df_scratch = pd.read_excel("./results/csr3_csr1_scratch2.xlsx", engine="openpyxl")
# csr3_csr1_transfer_1.xlsx, csr_incremental_results2, csr3_csr1_incremental_2
df_tran = pd.read_excel("./results/csr3_csr1_transfer_1.xlsx", engine="openpyxl")

for d in [('CSR3', 'CSR1')]:
    print(d)
    df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name=d[0], engine="openpyxl")
    df2 = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name= d[1], engine="openpyxl")

    x_axis = [x[6:8] for x in df_scratch.keys()]
    p_scratch = pmse(df_scratch, df2["FN"])
    m_scratch = mse(df_scratch, df2["FN"])
    p_tran = pmse(df_tran, df2["FN"])
    m_tran = mse(df_tran, df2["FN"])

    cutoff = 9 #len(p_incremental)
    pl = plt.plot(x_axis[:cutoff], p_tran[:cutoff], "silver", marker=markers[i],linestyle=linestyles[i%4], label = "AIL")
    pl2 = plt.plot(x_axis[:cutoff], p_scratch[:cutoff], "black", marker=markers[i],linestyle=linestyles[i%4], label = "IL")
    break
    i = i + 1
    
    
plt.xlabel("Percent of Data")
plt.ylabel("MSE")
plt.title("CSR3-CSR1")
plt.legend()
# plt.savefig("CSR3_CSR1_mse.jpg",dpi=300, bbox_inches = "tight" )
plt.show()

('CSR3', 'CSR1')
40
40
39
40
40
39
40
40
40
40
40
39
40
40
39
40
40
40


In [46]:
colors = ["silver", "black", "blue", "red", "green", "dimgrey"]

csr3_mse = [33.5274, 32.7185, 31.8467, 32.0178, 32.096, 32.8028, 33.8321, 35.6158, 37.8203, 38.2288, 40.926, 43.7721, 41.7924,
44.1037,
46.0917,
49.7141,
53.9997,
52.0667,
56.1043,
60.2474]
csr3_mse = csr3_mse[::-1]

d = "CSR3"
df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name=d, engine="openpyxl")
x_axis = list(range(len(list(range( int(0.8 * len(df)), len(df) -1))),0, -1))


activations = ["Elu", "ReLU", "GO-NN", "Sigmoid", "Tanh"]


for i in range(len(activations)):
#     if activations[i] == "go" or activations[i] == "relu":
#         continue
#     p = pmse(df_small[d][df_small[d].keys()[i*len(x_axis):(i + 1 )* len(x_axis) ]], df["FN"])
    m = mse(df_small[d][df_small[d].keys()[i*len(x_axis):(i + 1 )* len(x_axis) ]], df["FN"])
    
#     p = pmse(df_small_sys2[df_small_sys2.keys()[i *(int(len(df_small_sys2)*0.2)+1):(i+1)*(int(len(df_small_sys2)*0.2)+1) +1]], df_sys2["FN"])
#     m = mse(df_small_sys2[df_small_sys2.keys()[i *(int(len(df_small_sys2)*0.2)+1):(i+1)*(int(len(df_small_sys2)*0.2)+1) +1]], df_sys2["FN"])

    pl = plt.plot(x_axis[:len(x_axis)], m[:len(x_axis)], colors[i], marker="o", label=activations[i])


# go_small = pd.read_excel("./results/mle_small_steps.xlsx", engine="openpyxl")
    
    
# print(len(x_axis), len(go_small["PMSE"]) )
plt.plot(x_axis[:len(x_axis)], csr3_mse, colors[i+1], marker="o", label="GO-NHPP")

plt.xticks(x_axis[::2]+[1])
plt.gca().invert_xaxis()
plt.xlabel("Testing Size")
plt.ylabel("MSE")
plt.legend()
plt.savefig("CSR3_small_steps_mse.jpg",dpi=300, bbox_inches = "tight" )
plt.show()

Index(['CSR3_elu_83', 'CSR3_elu_84', 'CSR3_elu_85', 'CSR3_elu_86',
       'CSR3_elu_87', 'CSR3_elu_88', 'CSR3_elu_89', 'CSR3_elu_90',
       'CSR3_elu_91', 'CSR3_elu_92', 'CSR3_elu_93', 'CSR3_elu_94',
       'CSR3_elu_95', 'CSR3_elu_96', 'CSR3_elu_97', 'CSR3_elu_98',
       'CSR3_elu_99', 'CSR3_elu_100', 'CSR3_elu_101', 'CSR3_elu_102'],
      dtype='object')
1.934398412704468 1
2.143687725067139 2
2.236701965332031 3
3.771043777465821 4
3.782663345336914 5
4.200998306274414 6
7.662050724029541 7
7.987086296081544 8
8.335306167602539 9
9.495800971984865 10
11.37506008148193 11
12.16361808776856 12
14.13433933258057 13
15.38583374023437 14
15.45535278320312 15
15.8029203414917 16
15.94193649291992 17
16.12728691101074 18
19.13811683654785 19
19.57798767089844 20
20.01781463623047 21
22.60951614379883 22
22.95633316040039 23
23.32615470886231 24
26.8793773651123 25
28.03070068359375 26
28.26083374023437 27
28.30685615539551 28
28.81299209594726 29
30.03148841857911 30
30.46802902221679 31

64.36917877197266 63
64.717041015625 64
64.72540283203125 65
64.83394622802734 66
67.23817443847656 67
70.06565856933592 68
70.08937072753908 69
70.10516357421876 70
70.32227325439452 71
71.92818450927733 72
72.06890106201172 73
72.12749481201172 74
74.35493469238281 75
76.6031494140625 76
76.83367919921876 77
76.90665435791017 78
76.9834442138672 79
77.28662872314453 80
77.37868499755861 81
79.444580078125 82
79.64300537109375 83
85.69764709472655 84
86.8819122314453 85
86.88568115234375 86
89.75554656982423 87
89.79308319091798 88
89.86817169189452 89
90.40855407714844 90
90.51360321044923 91
90.72362518310545 92
92.50688171386717 93
92.75012207031251 94
93.1167221069336 95
96.4175567626953 96
97.20759582519533 97
98.27626800537108 98
98.47350311279298 99
98.48466491699217 100
101.5730209350586 101
101.7326278686524 102
101.9367446899414 103
102.3411560058594 104
2.411858081817627 1
2.6203453540802 2
2.712998628616333 3
4.240648746490479 4
4.252222537994385 5
4.668505191802979 6
8.10

88.44970703125 85
88.45355987548828 86
91.38594055175783 87
91.42435455322267 88
91.5011749267578 89
92.05419158935545 90
92.16170501708986 91
92.37675476074217 92
94.20395660400389 93
94.45339965820311 94
94.82948303222656 95
98.22042846679689 96
99.03334045410158 97
100.1337127685547 98
100.336898803711 99
100.348403930664 100
103.5335845947266 101
103.698371887207 102
103.9091491699219 103
104.3268432617188 104
1.489661574363708 1
1.705966711044311 2
1.802092432975769 3
3.387781620025634 4
3.399800777435304 5
3.832105875015258 6
7.40822219848633 7
7.744016647338867 8
8.103754997253418 9
9.302578926086424 10
11.24344635009766 11
12.05710029602051 12
14.08809471130371 13
15.37612533569336 14
15.44763088226318 15
15.80509376525879 16
15.94804000854492 17
16.13860702514648 18
19.2280387878418 19
19.67824935913086 20
20.12813568115234 21
22.77359008789062 22
23.12708091735839 23
23.50381469726563 24
27.07531929016113 25
28.18523406982422 26
28.40245819091798 27
28.44570541381835 28
28.91

19.48492050170898 2
19.5130615234375 3
19.97738265991211 4
19.98089981079101 5
20.10753250122071 6
21.1557731628418 7
21.25426483154297 8
21.35979461669922 9
21.71155166625976 10
22.28140258789062 11
22.52059555053711 12
23.11858749389649 13
23.49848556518555 14
23.51958847045899 15
23.6251220703125 16
23.66732788085937 17
23.72360992431641 18
24.63818359375 19
24.77185440063476 20
24.90552139282226 21
25.6934585571289 22
25.79898834228516 23
25.91154861450196 24
26.99496841430665 25
27.34672546386719 26
27.41707992553711 27
27.43114852905273 28
27.58592224121094 29
27.95878601074218 30
28.09245300292968 31
28.50049209594726 32
28.64119720458984 33
28.66230392456054 34
29.30953979492188 35
29.3447151184082 36
29.80903625488281 37
31.84220123291016 38
31.8633041381836 39
31.9266242980957 40
32.01104736328126 41
32.13767623901368 42
32.20099639892578 43
32.72863388061523 44
32.83416366577149 45
34.88139724731446 46
36.37285232543945 47
36.40099334716797 48
36.4361686706543 49
38.60300064

23.15900802612305 24
26.53422737121582 25
27.63007736206055 26
27.84924697875976 27
27.89308547973633 28
28.37525558471679 29
29.53685760498047 30
29.95327949523926 31
31.2244644165039 32
31.66280746459961 33
31.72855758666991 34
33.74492263793946 35
33.85450744628906 36
35.30103302001954 37
41.63504791259766 38
41.70079803466798 39
41.89805221557618 40
42.16105651855469 41
42.55555725097657 42
42.75281524658204 43
44.39658737182618 44
44.725341796875 45
49.07036972045899 46
49.98216247558594 47
49.99937057495117 48
50.02087020874023 49
51.34555435180665 50
52.50249099731446 51
53.68954467773436 52
53.69384384155274 53
55.41421127319337 54
56.67867279052735 55
57.65497970581053 56
58.16248703002928 57
58.21839523315431 58
58.42053985595704 59
58.80331802368163 60
59.84413909912108 61
60.26993179321288 62
62.88057708740234 63
63.23755645751952 64
63.24615859985351 65
63.35798263549805 66
65.87831115722656 67
68.92335510253906 68
68.94915771484376 69
68.96636199951173 70
69.2029113769531

...

28.53694915771484 28
29.05942535400391 29
30.29623413085937 30
30.73154449462891 31
32.03183364868164 32
32.46974563598634 33
32.53495788574219 34
34.47140884399414 35
34.57304382324218 36
35.87844085693359 37
40.78413009643555 38
40.8282127380371 39
40.95965194702148 40
41.13302230834961 41
41.38904571533203 42
41.51526260375976 43
42.52156829833985 44
42.71330642700195 45
45.87099075317383 46
47.61708068847656 47
47.646484375 48
47.68308258056641 49
49.64816665649415 50
51.05805206298827 51
52.39271545410156 52
52.39751434326173 53
54.37664413452148 54
55.95694732666017 55
57.24641418457032 56
57.93004226684572 57
58.00558090209961 58
58.27873229980469 59
58.79521179199217 60
60.17784118652344 61
60.72762298583983 62
63.81756591796874 63
64.20465087890625 64
64.2138900756836 65
64.33372497558594 66
66.92530059814453 67
70.04464721679688 68
70.07183837890624 69
70.08998107910158 70
70.3402099609375 71
72.24279022216797 72
72.41342163085938 73
72.48466491699219 74
75.23746490478516 75


48.02442169189454 49
50.0389175415039 50
51.49639511108398 51
52.86904144287109 52
52.87393951416015 53
54.86283493041994 54
56.40678787231445 55
57.64763259887695 56
58.30234909057616 57
58.37465286254883 58
58.63616180419923 59
59.13100051879881 60
60.46293258666992 61
60.99757003784178 62
64.09296417236328 63
64.49353790283203 64
64.50314331054686 65
64.62774658203125 66
67.3737258911133 67
70.69610595703125 68
70.72461700439453 69
70.74362182617188 70
71.00521850585936 71
72.95281982421875 72
73.12346649169923 73
73.1944808959961 74
75.83682250976564 75
78.28729248046875 76
78.52257537841795 77
78.59640502929688 78
78.67378997802733 79
78.97605133056642 80
79.06680297851564 81
80.99195098876952 82
81.16721343994142 83
86.30471038818358 84
87.36659240722658 85
87.37004089355467 86
90.12263488769533 87
90.16030883789061 88
90.23577117919923 89
90.78335571289061 90
90.89066314697264 91
91.10605621337892 92
92.97070312499999 93
93.22874450683595 94
93.61860656738283 95
97.1109619140625

16.06472015380859 16
16.20735740661621 17
16.39770126342773 18
19.50870704650879 19
19.96476173400879 20
20.42075920104981 21
23.09957122802735 22
23.45613861083984 23
23.83565521240235 24
27.42722320556641 25
28.56160926818848 26
28.78619194030762 27
28.83101081848145 28
29.32185554504394 29
30.4870433807373 30
30.89843177795409 31
32.13208770751954 32
32.54937744140626 33
32.61159133911133 34
34.47056579589844 35
34.56877899169922 36
35.83678817749024 37
40.73371505737305 38
40.77883529663085 39
40.91350936889648 40
41.0914421081543 41
41.35485458374023 42
41.48500442504883 43
42.5295753479004 44
42.73002243041993 45
46.09745788574218 46
48.00177001953124 47
48.03390884399413 48
48.07390213012695 49
50.19348907470702 50
51.62281417846679 51
52.85528564453124 52
52.85951232910157 53
54.55206298828124 54
55.96268463134764 55
57.23693084716798 56
57.96217346191406 57
58.0442008972168 58
58.3436393737793 59
58.92012405395506 60
60.49638748168945 61
61.11908340454101 62
64.27746582031249 

72.14481353759766 73
72.22333526611328 74
75.16658020019533 75
77.79170227050783 76
78.03450775146486 77
78.11038208007814 78
78.1897201538086 79
78.4979476928711 80
78.59001159667969 81
80.49308013916017 82
80.66209411621092 83
85.17637634277345 84
85.94879913330077 85
85.9511947631836 86
87.6565170288086 87
87.67710113525392 88
87.71812438964844 89
88.00785064697266 90
88.06300354003908 91
88.17215728759764 92
89.03631591796876 93
89.14540100097655 94
89.30580139160156 95
90.5373077392578 96
90.77800750732422 97
91.07314300537111 98
91.12396240234375 99
91.12680053710936 100
91.78943634033203 101
91.81760406494139 102
91.8528289794922 103
91.92012786865234 104
2.075296640396118 1
2.260296583175659 2
2.342721939086914 3
3.7203688621521 4
3.730939626693726 5
4.112447261810303 6
7.360411643981932 7
7.673428535461425 8
8.01020336151123 9
9.142879486083986 10
11.00821018218994 11
11.80118465423584 12
13.80518913269043 13
15.09119606018066 14
15.16286849975586 15
15.52156066894531 16
15.66

78.79203796386719 78
78.86753845214842 79
79.15878295898438 80
79.2450637817383 81
80.95377349853517 82
81.09880065917967 83
85.07997131347655 84
85.98590850830078 85
85.98897552490233 86
88.77361297607422 87
88.81624603271486 88
88.9019775390625 89
89.53582000732423 90
89.6621856689453 91
89.91764831542967 92
92.17141723632811 93
92.47964477539061 94
92.93876647949219 95
96.3794631958008 96
96.95619964599611 97
97.58838653564453 98
97.68820953369142 99
97.69371032714844 100
98.73932647705077 101
98.77354431152342 102
98.81526184082033 103
98.89157867431639 104
0.2664791345596313 1
0.5133013725280762 2
0.6229716539382935 3
2.428084373474122 4
2.441725254058838 5
2.932416915893555 6
6.959744453430174 7
7.33440637588501 8
7.734994411468507 9
9.063813209533691 10
11.19316864013672 11
12.07751083374024 12
14.26132488250733 13
15.62700939178466 14
15.70235729217529 15
16.07825660705566 16
16.22821426391602 17
16.42779731750488 18
19.61091041564941 19
20.06618499755859 20
20.518798828125 21


In [5]:
m = mse(df_dlb, df["FN"])
pm = pmse(df_dlb, df["FN"], split=0.8)
ma = mape(df_dlb, df["FN"])
p = pp(df_dlb, df["FN"], split=0.8)
pr = prr(df_dlb, df["FN"], split=0.8)
for k in range(0, len(df_dlb.keys())-1):
    print("{} & {:.3f} & {:.3f} & {:.3f} & {:.3} & {:.3}\\\\".format(df_dlb.keys()[k+1], m[k], ma[k], pm[k],pr[k], p[k]))
    
    
    
    

sys2_80_20 & 1618.417 & 60.095 & 3011.780 & 19.0 & 5.67\\
sys2_90_10 & 1658.923 & 62.261 & 3174.081 & 20.6 & 5.94\\
sys1_90_10 & 10.693 & 15.096 & 22.793 & 0.0444 & 0.0403\\
sys1_80_20 & 24.467 & 11.902 & 105.137 & 0.225 & 0.187\\


In [61]:
df_graphing = df_80_20["CSR3"]
df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR3", engine="openpyxl")
["elu", "relu", "go", "sigmoid", "tanh"]

df_go = pd.read_excel("./results/go_csr3_results.xlsx", engine="openpyxl")

from matplotlib import lines #marker=markers[4]
from matplotlib import markers
linestyles = list(lines.lineStyles.keys())
markers = list(markers.MarkerStyle.markers.keys())
one = plt.plot(df["FT"], df_graphing[df_graphing.keys()[0]], "blue" ,linestyle=linestyles[1], label = "Elu")
two = plt.plot(df["FT"], df_graphing[df_graphing.keys()[1]], "red",linestyle=linestyles[2], label = "ReLU")
go = plt.plot(df["FT"],  df_graphing[df_graphing.keys()[2]], "dimgray", linestyle=linestyles[0], label="GO-NN")
three = plt.plot(df["FT"], df_graphing[df_graphing.keys()[3]], "silver",linestyle=linestyles[1], label = "Sigmoid")
four = plt.plot(df["FT"], df_graphing[df_graphing.keys()[4]], "green", linestyle=linestyles[3], label = "Tanh")



go = plt.plot(df["FT"], df_go["GO FT"], "brown", linestyle=linestyles[0], label = "GO-NHPP")


actual = plt.step(df["FT"], df["FN"], "black", linestyle=linestyles[0], label="CSR3")

plt.axvline(x=df["FT"][int(len(df["FT"])*0.8)], color="black",linestyle=linestyles[1])
# plt.xticks(range(0, 17000, 2000))
# plt.ticklabel_format(style='sci', axis='x', scilimits=(3,3))

plt.xlabel("Testing Time")
plt.ylabel("Cumulative Failures")
plt.grid()
plt.legend()
# plt.savefig("CSR3_80_20_activations.jpg",dpi=300, bbox_inches = "tight")

In [30]:


df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR3", engine="openpyxl")
df_dss = pd.read_excel("./results/dss_80_20.xlsx", engine="openpyxl")


df_graphing = df_80_20["CSR3"]

df_graphing["go-nhpp"] = df_go["GO FT"]
df_graphing["dss-nhpp"] = df_dss["DSS FT"]
# print(df_graphing.keys())

p = pmse(df_graphing, df["FN"])
m = mse(df_graphing, df["FN"])
ma = mape(df_graphing, df["FN"])
ppp = pp(df_graphing, df["FN"])
pr = prr(df_graphing, df["FN"])


for i in range(0,7):

    print("& {:.3f} & {:.3f} & {:.3f} & {:.3f} & {:.3f}\\\\".format( m[i], ma[i],p[i], pr[i], ppp[i] ))




# plt.xlabel("Testing Time")
# plt.ylabel("Cumulative Failures")

plt.show()

& 4.180 & 4.558 & 15.283 & 0.034 & 0.037\\
& 5.936 & 6.343 & 22.380 & 0.047 & 0.053\\
& 11.481 & 7.585 & 44.733 & 0.085 & 0.099\\
& 18.381 & 7.780 & 85.900 & 0.238 & 0.181\\
& 24.812 & 6.249 & 118.503 & 0.345 & 0.252\\
& 38.228 & 17.270 & 14.296 & 0.034 & 0.033\\
& 111.207 & 32.626 & 119.956 & 0.349 & 0.256\\


In [106]:
from matplotlib import lines #marker=markers[4]
from matplotlib import markers
linestyles = list(lines.lineStyles.keys())
markers = list(markers.MarkerStyle.markers.keys())
go = plt.plot(df["FT"],  df0[df0.keys()[1]], "dimgray", linestyle=linestyles[0], label="GO-NN")
one = plt.plot(df["FT"], df0[df0.keys()[2]], "blue" ,linestyle=linestyles[1], label = "Elu")
two = plt.plot(df["FT"], df0[df0.keys()[3]], "red",linestyle=linestyles[2], label = "ReLU")
three = plt.plot(df["FT"], df0[df0.keys()[4]], "green", linestyle=linestyles[3], label = "Tanh")
four = plt.plot(df["FT"], df0[df0.keys()[5]], "silver",linestyle=linestyles[1], label = "Sigmoid")

go = plt.plot(df["FT"], df_go[df_go.keys()[-1]], "brown", linestyle=linestyles[0], label = "GO-NHPP")



actual = plt.step(df["FT"], df["FN"], "black", linestyle=linestyles[0], label="SYS1")
plt.axvline(x=df["FT"][int(len(df["FT"])*0.8)], color="black",linestyle=linestyles[1])
plt.xticks(range(0, 100000, 10000))
plt.ticklabel_format(style='sci', axis='x', scilimits=(3,3))

plt.xlabel("Testing Time")
plt.ylabel("Cumulative Failures")
plt.grid()
plt.legend()
# plt.savefig("sys1_80_20_activations.jpg",dpi=300, bbox_inches = "tight")

In [26]:
df_res = pd.read_excel("./results/all_results_stats.xlsx", sheet_name="CSR3, CSR2", engine="openpyxl")
df_res

,pmse incremental,mse incremental,pmse scratch,mse scratch,pmse transfer,mse transfer
0,17.589163,11.602655,13.965151,9.178688,13.963650,9.120257
1,13.491528,11.343395,22.704537,8.278739,12.326058,4.554018
2,7.824599,7.791955,24.167051,9.039079,3.967663,4.116721
3,4.656298,5.946478,6.157064,4.513433,9.601559,4.114385
4,376.066496,84.020170,150.623615,28.289487,206.378555,37.662076
5,228.237506,39.858996,529.099873,81.687798,551.457611,84.606060
6,732.809374,97.320963,34.257300,9.264816,56.672182,10.369622
7,59211.063799,6640.938759,95.301610,15.612986,5.109391,6.355268
8,802.889620,97.391014,194.381766,25.040231,255.476550,31.789576


In [29]:
x_axis = [x for x in range(1,10)]
plt.plot(x_axis, df_res["pmse scratch"], label = "IL")
plt.plot(x_axis, df_res["pmse transfer"], label="AIL")
plt.xlabel("Percent of Data")
plt.ylabel("PMSE")
plt.legend()
# plt.savefig("csr3_csr1_pmse_final.jpg",dpi=300, bbox_inches = "tight" )

plt.show()

In [8]:
# p_incremental = {}
# m_incremental= {}
# p_scratch= {}
# m_scratch= {}
# p_tran= {}
# m_tran= {}

# results = {}

# for d in datasets:#, "relu", "sigmoid", "tanh"]:
#     df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name=d[0], engine="openpyxl")
#     df_sys2 = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name= d[1], engine="openpyxl")


#     df_incremental = df_inc[d]
#     df_scratch = df_scra[d]
#     df_trans = df_tran[d]
    
#     df_temp = pd.DataFrame()

#     df_temp["pmse incremental"] = pmse(df_incremental, df_sys2["FN"])
#     df_temp["mse incremental"]= mse(df_incremental, df_sys2["FN"])
#     df_temp["pmse scratch"] = pmse(df_scratch,df_sys2["FN"])
#     df_temp["mse scratch"] = mse(df_scratch, df_sys2["FN"])
#     df_temp["pmse transfer"] = pmse(df_trans, df_sys2["FN"])
#     df_temp["mse transfer"] = mse(df_trans, df_sys2["FN"])
#     results[d] = df_temp.copy()

    
# with pd.ExcelWriter("./results/all_results_stats.xlsx") as ex:
#     for i in results.keys():
#         results[i].to_excel(ex, "{}, {}".format(i[0], i[1]), index=False)



In [77]:
for d in [('CSR3', 'CSR1')]:
    print(d)
    rnn = pd.read_excel("./results/rnn_results_stats.xlsx", sheet_name="{}, {}".format(d[0], d[1]), engine="openpyxl")
    nn = pd.read_excel("./results/all_results_stats.xlsx", sheet_name="{}, {}".format(d[0], d[1]), engine="openpyxl")
    x_axis = [x for x in range(10, 100, 10) ]
#     plt.plot(x_axis, nn["pmse scratch"], label="scratch")
#     plt.plot(x_axis, nn["pmse transfer"], label="transfer")
    for method in [ "incremental 2"]: # "scratch","incremental 1", "transfer",
        plt.plot(x_axis, rnn["pmse {}".format(method)], label="rnn {}".format(method))
    plt.xlabel("Percent of Data")
    plt.ylabel("PMSE")
    plt.title("{}, {}".format(d[0], d[1]))
    plt.legend()
#     plt.savefig("./figures/pmse_{}_{}.jpg".format(d[0], d[1]), dpi=300, bbox_inches = "tight" )
    plt.show()


('CSR3', 'CSR1')


In [53]:
len(df)- int(len(df)* 0.8)

21

In [115]:
#change df1 to df2 or df3 to change to different data set with different number of layers
df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR3", engine="openpyxl")



lr = [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.0075, 0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75]
nodes = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]


def pmse(y_predicted, y_actual, split = 0.8):
    pmse_list = []
#     for i in range(1, len(y_predicted.keys())):  
    pmse =0
    split = 0.8
    for a, b in zip(y_predicted, list(y_actual[int(split* len(y_actual)):])):
#         print(a,b)
        pmse += (a-b)**2
#         print(len(list(y_actual[int(split* len(y_actual)):])))
    pmse= pmse/len(list(y_actual[int(split* len(y_actual)):]))
    return pmse

# p = pmse(df_lr_nodes, df["FN"])
# m = mse(df_lr_nodes, df["FN"])

data = []
for j in range(len(lr)):
    for i in range(len(nodes)):
        data.append( [lr[j], nodes[i], pmse(df_lr_nodes[df_lr_nodes.keys()[ j*len(nodes) + i]], df["FN"])] )
        
        
        

In [ ]:
# df10 = pd.DataFrame()
# df10["lr"] = [item[0] for item in data]
# df10["nodes"] = [item[1] for item in data]
# df10["mse"] = [item[2] for item in data]
# df10["pmse"] = [item[3] for item in data]
# df10.to_csv("pretty_graph_data.csv")

In [13]:
# min([item[2] for item in data]) #3.203851303963939, 6.499054471972943
data

NameError: name 'data' is not defined

In [117]:
#creates and plots figure. Can save from interactive plot window. 
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_trisurf([item[0] for item in data], [item[1] for item in data], [item[2] for item in data], cmap=plt.get_cmap('gray'))
plt.xlabel("Learning rate")
plt.ylabel("Nodes")
plt.xlabel("PMSE")
plt.show()